# 目标检测与目标识别 深度学习

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageColor
import os, shutil

In [4]:
import cv2

In [5]:
#读入训练和测试图像
train_car_dir = 'G:/data/train/cars_train' #原始数据路径
test_car_dir = 'G:/data/test/cars_test'
validation_cars_dir = 'G:/data/validation/cars'
validation_dirall = 'G:/data/validation'
train_dirall = 'G:/data/train'
test_dirall = 'G:/data/test'

print('total training images:', len(os.listdir(train_car_dir)))#看看每个分组（训练/ 测试）中分别包含多少张图像
print('total testing images:', len(os.listdir(test_car_dir)))

total training images: 20
total testing images: 20


In [6]:
#卷积神经网络
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))#二分类问题,激活函数relo
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [7]:
#配置模型用于训练
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy',
                optimizer=optimizers.RMSprop(lr=1e-4),
                metrics=['acc'])

In [8]:
#数据预处理
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data = ImageDataGenerator(rescale=1./255)#将所有图像乘以1/255 缩放
test_data = ImageDataGenerator(rescale=1./255)#将所有图像乘以1/255 缩放

train_generator = train_data.flow_from_directory(
                                        train_dirall,#目标目录
                                        target_size=(150, 150),#将所有图像的大小调整为150×150
                                        batch_size=20,
                                        class_mode='binary')#因为使用了binary_crossentropy损失，所以需要用二进制标签
validation_generator = test_data.flow_from_directory(
                                        validation_dirall,
                                        target_size=(150, 150),
                                        batch_size=20,
                                        class_mode='binary')

Found 40 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(
train_generator,
steps_per_epoch=100,
epochs=10,
validation_data=validation_generator,
validation_steps=50)

Epoch 1/10
 73/100 [====================>.........] - ETA: 34s - loss: 0.3293 - acc: 0.8795

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()